In [1]:
# !ls data/

In [2]:
import pandas as pd
from collections import Counter
import os
import glob
import re
import shutil
from matplotlib import pyplot as plt
import numpy as np

In [3]:
def count_null(df):
    return df.isnull().sum()
def columns_fix(df):
    try:
        df_ouro.drop('REF_ANOMES', inplace=True, axis=1)
        df_ouro['pagina_dodf'] = df_ouro.pagina_dodf.str.replace(
            r'([^\d]+)',''
        )
    except:
        pass
    df_ouro.rename(str.lower, inplace=True, axis=1)
    df_ouro.rename(lambda x: x.replace('_dodf', ''), axis=1, inplace=True)
    df_ouro.rename(lambda x: x.replace('matricula', 'mat'), axis=1, inplace=True)

def postprocess(df):
    df['pagina'] = df.pagina.str.replace(r'([^\d]+)', '')
    to_drona_drop = [
        'cod_mat_sigrh',
        'nome_sigrh',
        'fund_legal',
        'cargo',
        'ato',
        
    ]
    df.dropna(subset=to_drona_drop, inplace=True)
    df['data'] = pd.to_datetime(df.data)

In [4]:
df_ouro = pd.read_csv(
    'data/aposentadoria-ouro/aposentaria-ouro-®.csv', sep='®',
    engine='python',
    index_col=False,
)
# count_null(df_ouro)
columns_fix(df_ouro)
postprocess(df_ouro)
df_ouro.columns

Index(['data', 'num', 'pagina', 'tipo', 'ato', 'cod_empresa', 'empresa_ato',
       'cod_mat_ato', 'cod_mat_sigrh', 'cpf', 'nome_ato', 'nome_sigrh',
       'cargo', 'classe', 'padrao', 'quadro', 'processo', 'fund_legal'],
      dtype='object')

In [5]:
matters = ['num','data', 'nome_ato',
        'ato', 'pagina',
        'cod_mat_ato',
        'cod_mat_sigrh', 'tipo', 'processo'
        ]
df_zip = df_ouro[matters]
df_zip[df_zip.cod_mat_sigrh == '01007238']
# df_zip.head()

,num,data,nome_ato,ato,pagina,cod_mat_ato,cod_mat_sigrh,tipo,processo
116,68,2018-04-10,CLAUDIA CRISTIANE CAVALCANTE AIELLO,CONCEDER APOSENTADORIA,26,1660377X,01007238,NORMAL,°00070.00011832/2018-38


In [6]:
df_zip.ato.value_counts(),df_zip.tipo.value_counts()

(CONCEDER APOSENTADORIA    4719
 APOSENTAR                  766
 Name: ato, dtype: int64,
 NORMAL          5442
 EDICAO EXTRA      42
 SUPLEMENTO         1
 Name: tipo, dtype: int64)

In [7]:
!ls

data	      extract-dodfminer.ipynb	      run-comparison.ipynb
date_num.csv  match-aposentadoria-ouro.ipynb


In [68]:
gp = df_zip.groupby(['data', 'num'])
ks=[i for i in dir(gp) if not i.startswith('_')]
sa = Counter([i[0] for i in gp.data.count().index])
date_num = gp.count().iloc[:, [0]].sort_values(
    by=['nome_ato', 'data', 'num'], ascending=False)

# Arquivo com datas e mnumeros dos DODFs pra baixar
# cada qual contendo 50+ ocorrencias
date_num = date_num[date_num.iloc[:, 0] > 50]
date_num.reset_index(inplace=True)
date_num = date_num.rename({'nome_ato':'qtd'}, axis=1)
date_num['data'] = date_num.data.map(lambda x: x.strftime('%d-%m-%Y'))
date_num.sort_values('data').to_csv('data/date_num.csv', index=False);
date_num

# print("TOTAL (>50):", date_num[date_num.nome_ato > 50].sum()[0],end=' ')


,data,num,qtd
0,05-09-2019,169,131
1,03-07-2018,124,116
2,08-06-2018,108,100
3,01-08-2019,144,99
4,18-06-2018,114,88
5,19-08-2019,156,87
6,01-10-2018,187,82
7,12-03-2019,47,81
8,25-02-2019,39,81
9,03-08-2018,147,80


In [8]:
df_a = []
df_sea = []
for i in glob.glob('data/extraido/*.csv'):
    f = i.split('/')[-1]
    df = pd.read_csv(i, index_col=False)
    df.drop('Unnamed: 0', axis=1, inplace=True)
    df['num_date'] = df.shape[0] * [re.search(r'(\d.+\d)(?=[.])',f).group()]
    if f.startswith('aposentadoria'):
        df_a.append(df)
    else:
        df_sea.append(df)
df_a = pd.concat(df_a)
df_sea = pd.concat(df_sea)